# Modularizing the Testbench

In [1]:
assert 4 == 5, "There are FOUR lights!"

AssertionError: There are FOUR lights!

In [2]:
from pyuvm import *
from proxy_pkg import *
import random

class AluProxy(uvm_component):
    """Provides the ALU Python operations"""
    @staticmethod
    def alu_op(A, B, op, inject_errors = False):
        """Python model of the TinyALU"""
        assert isinstance(op, Ops), "The tinyalu op must be of type ops"
        if op == Ops.ADD:
            result = A + B
        elif op == Ops.AND:
            result = A & B
        elif op == Ops.XOR:
            result = A ^ B
        elif op == Ops.MUL:
            result = A * B
        if inject_errors:
            time.sleep(0.1)
            if random.randint(0,3) == 0:
                result = result + 1
        return result

class Coverage(uvm_component):
    
    def end_of_elaboration_phase(self):
        self.cvg = set()
    
    def write(self, op):
        assert isinstance(op, Ops), "Coverage can only receive Ops"
        self.cvg.add(op)

    def check_phase(self):
        if len(set(Ops) - self.cvg) > 0:
            self.logger.error(f"Functional coverage error. Missed: {set(Ops)-self.cvg}")

class Scoreboard(uvm_component):  

    def end_of_elaboration_phase(self):
        self.results=[]

    def write(self, op_result):
        A, B, op, actual_result = op_result
        predicted_result = AluProxy.alu_op(A, B, op)
        self.results.append( (A, B, op, predicted_result, actual_result) )

    def check_phase(self):
        for A, B, op, predicted_result, actual_result in self.results:
            if predicted_result == actual_result:
                self.logger.info( f"PASSED: {A:02x} {op.name} {B:02x} ="
                                 f" {actual_result:04x}")
            else:
                self.logger.error(f"FAILED: {A:02x} {op.name} {B:02x} ="
                                  f" {actual_result:04x} expected {predicted_result:04x}")


class AluEnv(uvm_env):
    def build_phase(self):
        self.cvg = Coverage("cvg", self)
        self.scb = Scoreboard("scb", self)
        self.proxy = AluProxy("proxy", self)
    
    def run_phase(self):
        self.raise_objection()  ## You MUST raise an objection

        for _ in range(3):
            A = random.randrange(256)
            B = random.randrange(256)
            op = random.choice(list(Ops))
            self.cvg.write(op)
            actual_result = self.proxy.alu_op(A, B, op, inject_errors=True)
            result_tuple = ( A, B, op, actual_result)
            self.scb.write( result_tuple )
        self.drop_objection()  ## drop the objection to end

class AluTest(uvm_test):
    def build_phase(self):
        self.env = AluEnv("env", self)

uvm_root().run_test("AluTest")

ERROR: <ipython-input-2-a247dc4e850f>(36)[uvm_test_top.env.cvg]: Functional coverage error. Missed: {<Ops.ADD: 1>, <Ops.MUL: 4>}
ERROR: <ipython-input-2-a247dc4e850f>(54)[uvm_test_top.env.scb]: FAILED: 92 XOR 9b = 000a expected 0009
INFO: <ipython-input-2-a247dc4e850f>(51)[uvm_test_top.env.scb]: PASSED: 19 XOR 5b = 0042
INFO: <ipython-input-2-a247dc4e850f>(51)[uvm_test_top.env.scb]: PASSED: 90 AND 5d = 0010
